In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.stats as st
import seaborn as sns
from mpmath import *
from decimal import Decimal
from timeit import default_timer as timer
import json

import tools as tl
import benchmark_func as bf
from hyperheuristic import Hyperheuristic
from metaheuristic import Metaheuristic
from experiment import read_config_file

In [2]:
sns.set(context="paper", font_scale=1.8, palette="husl", style="ticks",
        rc={'font.family': 'serif', 'font.size': 18,
            "xtick.major.top": False, "ytick.major.right": False})
plt.rc('font', size=18) 

# Saving images flag
is_saving = True
show_plots = True
saving_format = 'png'

figures_folder = 'data_files/exp_figures/'
experiment_folder = 'data_files/exp_output'
ml_model_folder = "./data_files/ml_models_times/"
exp_label = 'default_nn_best_double_lstm'
bm_exp_label = 'basic_metaheuristic'

if is_saving:
    # Read (or create if so) a folder for storing images
    if not os.path.isdir(figures_folder):
        os.mkdir(figures_folder)

In [3]:
files = os.listdir(ml_model_folder)
files = [file for file in files if 'DS_S' not in file]

# Number of dimensions
dimensions = [2, 10, 30, 50]
num_dims = len(dimensions)

problems = list(set(file.split('-')[0] for file in files))
len(problems), dimensions

(119, [2, 10, 30, 50])

In [4]:
data = tl.read_json('./data_files/exp_output/'+exp_label+'.json')

In [5]:
def sort_dataset(dataset):
    # Sort the results of the 428 problems in lexicographically order
    list_pair_results = list(zip(dataset['problem'],
                                 dataset['dimensions'],
                                 dataset['results']))
    list_pair_results.sort()
    dataset['problem'], dataset['dimensions'], dataset['results'] = [], [], []
    for problem, dimension, result in list_pair_results:
        dataset['problem'].append(problem)
        dataset['dimensions'].append(dimension)
        dataset['results'].append(result)
        
def filter_by_dimensions(dataset):
    # Filter the dataset allowing only the dimensions from 'dimensions' list.
    # The usercase is for the basic metaheuristics that has results for extra 
    # dimensions out of scope of this paper.
    allowed_dim_inds = [index for d in dimensions for index in tl.listfind(dataset['dimensions'], d)]
    dict_filtered = {key: [val[x] for x in allowed_dim_inds] for key, val in dataset.items()}
    sort_dataset(dict_filtered)
    return dict_filtered


In [6]:
# Read operators and find their alias
collections = ['default.txt', 'basicmetaheuristics.txt']

encoded_heuristic_space = dict()
for collection_file in collections:
    with open('./collections/' + collection_file, 'r') as operators_file:
        encoded_heuristic_space[collection_file] = [eval(line.rstrip('\n')) for line in operators_file]

# Search operator aliases
perturbator_alias = {
    'random_search': 'RS',
    'central_force_dynamic': 'CF',
    'differential_mutation': 'DM',
    'firefly_dynamic': 'FD',
    'genetic_crossover': 'GC',
    'genetic_mutation': 'GM',
    'gravitational_search': 'GS',
    'random_flight': 'RF',
    'local_random_walk': 'RW',
    'random_sample': 'RX',
    'spiral_dynamic': 'SD',
    'swarm_dynamic': 'PS'}

selector_alias = {'greedy': 'g', 'all': 'd', 'metropolis': 'm', 'probabilistic': 'p'}

operator_families = {y: i for i, y in enumerate(sorted([x for x in perturbator_alias.values()]))}

# Pre-build the alias list
heuristic_space = dict()
for collection_file in collections:
    if all(isinstance(x, tuple) for x in encoded_heuristic_space[collection_file]):
        heuristic_space[collection_file] = [perturbator_alias[x[0]] + selector_alias[x[2]]
                                            for x in encoded_heuristic_space[collection_file]]


In [7]:
# Read basic metaheuristics
basic_mhs_collection = encoded_heuristic_space['basicmetaheuristics.txt']

# Read basic metaheuristics cardinality
basic_mhs_cadinality = [1 if isinstance(x, tuple) else len(x) for x in basic_mhs_collection]

# Load data from basic metaheuristics
basic_mhs_data = filter_by_dimensions(tl.read_json(f'{experiment_folder}/basic-metaheuristics-data_v2.json'))

# Mathematical attributes
chosen_categories = ['Differentiable', 'Unimodal']
case_label = 'DU'

# Call the problem categories
problem_features = bf.list_functions(fts=chosen_categories)
categories = sorted(set([problem_features[x]['Code'] for x in basic_mhs_data['problem']]), reverse=True)


In [8]:
# Retrive the results per experiment
data_frame = filter_by_dimensions(data)

# Summary
id = 'dlstm'
data_info = {
    'Dim': [int(x) for x in data_frame['dimensions']],
    'Pop': 30,
    'Problem': data_frame['problem'],
    'Cat': [problem_features[x]['Code'] for x in data_frame['problem']],
    'mhs': [x['encoded_solution'] for x in data_frame['results']],
    'hist': [x['hist_fitness'] for x in data_frame['results']]
}

In [9]:
def get_correct_id(problem_name, dimension):
  for id in range(len(data_info['Problem'])):
    correct_problem = data_info['Problem'][id] == problem_name
    correct_dimension = data_info['Dim'][id] == dimension
    if correct_problem and correct_dimension:
      return id
  return -1

In [10]:
"""
exp_config, hh_config, prob_config = read_config_file(exp_label)
# Message to print and to store in folders
problem_name = 'Sphere'
dimension = 2
label = '{}-{}D-{}'.format(problem_name, dimension, exp_label)

# Get and format the problem
# problem = eval('bf.{}({})'.format(problem, dimension))
problem = bf.choose_problem(problem_name, dimension)

problem.set_offset_domain(2)
problem.set_offset_function(3)
problem.set_scale_domain(1.5)
problem.set_scale_function(4)
problem.set_noise_type('normal')
problem.set_noise_level(2)

problem_to_solve = problem.get_formatted_problem(True, ['Differentiable', 'Separable', 'Unimodal'])
from optproblems.cec2005 import F1
fun = F1(2, None)
problem_to_solve['function'] = fun

# Call the hyper-heuristic object
hh = Hyperheuristic(heuristic_space=exp_config['heuristic_collection_file'],
                        problem=problem_to_solve, parameters=hh_config,
                        file_label=label, weights_array=None)
"""

"\nexp_config, hh_config, prob_config = read_config_file(exp_label)\n# Message to print and to store in folders\nproblem_name = 'Sphere'\ndimension = 2\nlabel = '{}-{}D-{}'.format(problem_name, dimension, exp_label)\n\n# Get and format the problem\n# problem = eval('bf.{}({})'.format(problem, dimension))\nproblem = bf.choose_problem(problem_name, dimension)\n\nproblem.set_offset_domain(2)\nproblem.set_offset_function(3)\nproblem.set_scale_domain(1.5)\nproblem.set_scale_function(4)\nproblem.set_noise_type('normal')\nproblem.set_noise_level(2)\n\nproblem_to_solve = problem.get_formatted_problem(True, ['Differentiable', 'Separable', 'Unimodal'])\nfrom optproblems.cec2005 import F1\nfun = F1(2, None)\nproblem_to_solve['function'] = fun\n\n# Call the hyper-heuristic object\nhh = Hyperheuristic(heuristic_space=exp_config['heuristic_collection_file'],\n                        problem=problem_to_solve, parameters=hh_config,\n                        file_label=label, weights_array=None)\n"

In [11]:
def get_best_mhs(problem_name, dimension):
  id = get_correct_id(problem_name, dimension)
  hists = [historial[-1] for historial in data_info['hist'][id]]
  optimal_position = np.argmin(hists)
  return data_info['mhs'][id][optimal_position]

In [12]:
exp_config, hh_config, prob_config = read_config_file(exp_label)

def get_evaluation(problem_name, dimension, problem_to_solve):
  label = '{}-{}D-{}'.format(problem_name, dimension, exp_label)
  hh = Hyperheuristic(heuristic_space=exp_config['heuristic_collection_file'],
                      problem=problem_to_solve, parameters=hh_config,
                      file_label=label, weights_array=None)
  mhs =  get_best_mhs(problem_name, dimension)
  hh.parameters['num_replicas'] = 30
  hh.num_iterations = 1
  start_time = timer()
  results = hh.evaluate_candidate_solution(mhs)
  end_time = timer() - start_time
  return (end_time, results)

def get_from_trained_model(problem_name, dimension, problem_to_solve):
  label = '{}-{}D-{}'.format(problem_name, dimension, exp_label)
  hh = Hyperheuristic(heuristic_space=exp_config['heuristic_collection_file'],
                      problem=problem_to_solve, parameters=hh_config,
                      file_label=label, weights_array=None)
  start_time = timer()
  hh.parameters['num_replicas'] = 30
  results = hh._solve_neural_network()
  end_time = timer() - start_time
  return (end_time, results)
  

bm_exp_config, bm_hh_config, bm_prob_config = read_config_file(bm_exp_label)
def get_bm_evaluation(func_name, dimension, problem_to_solve):
  label = '{}-{}D-{}'.format(func_name, dimension, bm_exp_label)
  hh = Hyperheuristic(heuristic_space=bm_exp_config['heuristic_collection_file'],
                      problem=problem_to_solve, parameters=bm_hh_config,
                      file_label=label, weights_array=None)
  results = hh.basic_metaheuristics()
  print(f'{func_name} with bmhs done!')
  return results

In [13]:
cec_relation = {
  'F1': ['Sphere'],
  'F2': ['Schwefel12'],# 'Schwefel'],
  'F4': ['Schwefel'],# 'Schwefel12'],
  'F5': ['Schwefel226'],# 'Schwefel'],
  'F6': ['Rosenbrock'],
  'F7': ['Griewank'],
  'F8': ['Ackley1'],# 'Ackley4'],
  'F9': ['Rastrigin'],
  'F10': ['Rastrigin'],
  'F11': ['Weierstrass'],
  'F13': ['Rosenbrock', 'Griewank'],
  'F14': ['SchafferN1']#, 'SchafferN2', 'SchafferN3', 'SchafferN4', 'SchafferN6'],
}

cec_boundaries = {
  'F1': (-100, 100),
  'F2': (-100, 100),
  'F4': (-100, 100),
  'F5': (-100, 100),
  'F6': (-100, 100),
  'F7': (0, 600),
  'F8': (-32, 32),
  'F9': (-5, 5),
  'F10': (-5, 5),
  'F11': (-0.5, 0.5),
  'F13': (-3, 1),
  'F14': (-100, 100),
}

cec_constrained = {
  'F1': True,
  'F2': True,
  'F4': True,
  'F5': True,
  'F6': True,
  'F7': False,
  'F8': True,
  'F9': True,
  'F10': True,
  'F11': True,
  'F13': True,
  'F14': True,
}

In [ ]:
from optproblems.cec2005 import *

results_dlstm = {
  'func_name': [],
  'base_func': [],
  'dimension': [],
  'performance': [],
  'fitness': [],
  'time': [],
}
def get_cec_function_formatted(f_name, dimension):
  f_initializer = eval(f_name)
  if f_name == 'F7':
    f_function = f_initializer(dimension)
  else:
    f_function = f_initializer(dimension, None)
  pre_bound = cec_boundaries[f_name]
  boundaries = ([pre_bound[0]]*dimension, [pre_bound[1]]*dimension)
  constrained = cec_constrained[f_name]
  problem_to_solve = {
    'function': f_function, 
    'boundaries': boundaries, 
    'is_constrained': constrained
  }
  return problem_to_solve
  
for f_name, bf_names in cec_relation.items():
  for dimension in dimensions:
    
    cec_function = get_cec_function_formatted(f_name, dimension)
    """
    label = '{}-{}D-{}_CEC05'.format(f_name, dimension, exp_label)
    hh = Hyperheuristic(heuristic_space=exp_config['heuristic_collection_file'],
                        problem=cec_function, parameters=hh_config,
                        file_label=label, weights_array=None)
    hh.parameters['verbose']=True
    hh.parameters['model_params']['epochs'] = 3
    start_time = timer()
    res = hh._solve_neural_network()
    end_time = timer() - start_time
    final_fit = [x[-1] for x in res[0]]
    performance_mhs = np.median(final_fit) + st.iqr(final_fit)
    results_dlstm['func_name'].append(f_name)
    results_dlstm['base_func'].append('neural network')
    results_dlstm['dimension'].append(dimension)
    results_dlstm['performance'].append(performance_mhs)
    results_dlstm['fitness'].append(res[0])
    results_dlstm['time'].append(end_time)    
    """
    
    for problem_name in bf_names:

      end_time, res = get_evaluation(problem_name, dimension, cec_function)
      performance_mhs = res[0]
      historical_replicas = res[1]['historical']
      hist_fitness = [[int(fitness) for fitness in replica['fitness']] for replica in historical_replicas]
      best_fitness = min(x[-1] for x in hist_fitness)
      
      results_dlstm['func_name'].append(f_name)
      results_dlstm['base_func'].append(problem_name)
      results_dlstm['dimension'].append(dimension)
      results_dlstm['performance'].append(performance_mhs)
      results_dlstm['fitness'].append(hist_fitness)
      results_dlstm['time'].append(end_time)
      print(f_name, problem_name, dimension, best_fitness, end_time)


In [45]:
import json
json_object = json.dumps(results_dlstm, indent=4)
with open('dlstm_cec2005_results.json', 'w') as outfile:
  outfile.write(json_object)

In [15]:

results_dlstm_from_trained = {
  'func_name': [],
  'base_func': [],
  'dimension': [],
  'performance': [],
  'fitness': [],
  'time': [],
}
for f_name, bf_names in cec_relation.items():
  for dimension in dimensions:
    
    cec_function = get_cec_function_formatted(f_name, dimension)
    
    for problem_name in bf_names:

      end_time, res = get_from_trained_model(problem_name, dimension, cec_function)
      hist_fitness = res[0]
      last_fitness = [x[-1] for x in hist_fitness]
      performance_mhs = np.median(last_fitness) + st.iqr(last_fitness)
      best_fitness = min(x[-1] for x in hist_fitness)
      
      results_dlstm_from_trained['func_name'].append(f_name)
      results_dlstm_from_trained['base_func'].append(problem_name)
      results_dlstm_from_trained['dimension'].append(dimension)
      results_dlstm_from_trained['performance'].append(performance_mhs)
      results_dlstm_from_trained['fitness'].append(hist_fitness)
      results_dlstm_from_trained['time'].append(end_time)
      print(f_name, problem_name, dimension, best_fitness, end_time)

json_object = json.dumps(results_dlstm_from_trained, indent=4)
with open('dlstm_from_trained_model_cec2005_results.json', 'w') as outfile:
  outfile.write(json_object)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

F1 Sphere 2 -450.0 350.627701375
F1 Sphere 10 -449.99994329270777 1079.1201958340002
F1 Sphere 30 -285.2369971899556 1181.2999925000001
F1 Sphere 50 2669.794071543517 1055.4062423330001
F2 Schwefel12 2 -450.0 324.5715741250001
F2 Schwefel12 10 -449.9939667316279 985.7433370000003
F2 Schwefel12 30 5921.170942323744 1064.2448900829995
F2 Schwefel12 50 30701.26893404057 1041.320855041
F4 Schwefel 2 -450.0 374.3621294170007
F4 Schwefel 10 -449.3130685003694 921.1202790409998
F4 Schwefel 30 3247.953168810068 803.646753957999
F4 Schwefel 50 38407.71438411115 658.7953004580013
F5 Schwefel226 2 -310.0 31.786186624998663
F5 Schwefel226 10 -309.8609145748269 738.7257363749995
F5 Schwefel226 30 4538.346232982845 953.9078023330003
F5 Schwefel226 50 14578.926887018019 867.5415054589994
F6 Rosenbrock 2 390.0000000001307 281.33045458299966
F6 Rosenbrock 10 392.92289281193615 927.9724528340012
F6 Rosenbrock 30 3539666.778

In [40]:
len(results_dlstm_from_trained['func_name'])

24

In [16]:
results_bmhs = {
  'func_name': [],
  'base_func': [],
  'dimension': [],
  'performance': [],
  'fitness': [],
  'time': [],
}

for f_name in cec_relation.keys():
    for dimension in dimensions:

      cec_function = get_cec_function_formatted(f_name, dimension)
      
      res = get_bm_evaluation(f_name, dimension, cec_function)

      for id, operators in enumerate(encoded_heuristic_space['basicmetaheuristics.txt']):
        problem_name = operators[0]
        if isinstance(problem_name, tuple):
          problem_name = problem_name[0]
        performance_mhs = res['performance'][id]
        hist_fitness = res['fitness'][id]
        end_time = res['time'][id]
        
        results_bmhs['func_name'].append(f_name)
        results_bmhs['base_func'].append(problem_name)
        results_bmhs['dimension'].append(dimension)
        results_bmhs['performance'].append(performance_mhs)
        results_bmhs['fitness'].append(hist_fitness)
        results_bmhs['time'].append(end_time)
        best_fitness = min(x[-1] for x in hist_fitness)
        print(f_name, problem_name, dimension, performance_mhs, end_time)



F1-2D-basic_metaheuristic :: BasicMH 1 of 66, Perf: -441.15853476378345
F1-2D-basic_metaheuristic :: BasicMH 2 of 66, Perf: -449.9925349947022
F1-2D-basic_metaheuristic :: BasicMH 3 of 66, Perf: -449.96574428837033
F1-2D-basic_metaheuristic :: BasicMH 4 of 66, Perf: -448.18774519300797
F1-2D-basic_metaheuristic :: BasicMH 5 of 66, Perf: -449.99702266725376
F1-2D-basic_metaheuristic :: BasicMH 6 of 66, Perf: -449.9792900000848
F1-2D-basic_metaheuristic :: BasicMH 7 of 66, Perf: -448.7380751434638
F1-2D-basic_metaheuristic :: BasicMH 8 of 66, Perf: -449.99849504575815
F1-2D-basic_metaheuristic :: BasicMH 9 of 66, Perf: -449.9770937823502
F1-2D-basic_metaheuristic :: BasicMH 10 of 66, Perf: -446.1479053889491
F1-2D-basic_metaheuristic :: BasicMH 11 of 66, Perf: -449.9997829836682
F1-2D-basic_metaheuristic :: BasicMH 12 of 66, Perf: -449.99657162322734
F1-2D-basic_metaheuristic :: BasicMH 13 of 66, Perf: -446.7810700470935
F1-2D-basic_metaheuristic :: BasicMH 14 of 66, Perf: -442.523210124

In [17]:
json_object = json.dumps(results_bmhs, indent=4)
with open('bmhs_cec2005_results.json', 'w') as outfile:
  outfile.write(json_object)

In [16]:

json_object = json.dumps(results_dlstm_from_trained, indent=4)
with open('dlstm_from_trained_model_cec2005_results.json', 'w') as outfile:
  outfile.write(json_object)